# BayesianSearchCV

1. Apa itu hyperparameter tuning?
> Hyperparameter tuning adalah proses mencari konfigurasi terbaik dari parameter model machine learning yang tidak dapat dipelajari langsung dari data. Contohnya meliputi learning rate, jumlah pohon pada Random Forest, atau jumlah hidden layer pada jaringan saraf tiruan. Tujuan dari hyperparameter tuning adalah untuk memperoleh performa model yang lebih optimal.

2. Jelaskan metode BayesSearchCV!
> BayesSearchCV adalah metode pencarian hyperparameter yang menggunakan pendekatan Bayesian Optimization dengan membungkus proses Cross Validation. Tidak seperti GridSearchCV yang mencoba semua kombinasi atau Random Search yang memilih secara acak, BayesSearchCV lebih efisien karena memanfaatkan informasi dari percobaan sebelumnya untuk memperkirakan kombinasi hyperparameter selanjutnya yang lebih menjanjikan.

3. Bagaimana cara kerja BayesSearchCV?
> - Pada awalnya, BayesSearchCV mencoba beberapa kombinasi hyperparameter secara acak.

> - Selanjutnya, dibuat model probabilistik yang memperkirakan hubungan antara hyperparameter dan performa model.

> - Berdasarkan model perkiraan tersebut, BayesSearchCV memilih kombinasi hyperparameter baru yang diprediksi lebih menjanjikan.

> - Proses ini diulangi hingga jumlah iterasi yang ditentukan tercapai.

> - Akhirnya, diperoleh kombinasi hyperparameter terbaik berdasarkan hasil evaluasi.

4. Apa kelebihan BayesSearchCV dibandingkan metode hyperparameter tuning lainnya?
> - Efisien waktu, karena tidak perlu mencoba semua kombinasi seperti GridSearch.

> - Lebih cerdas dibanding Random Search, sebab BayesSearchCV menggunakan informasi dari percobaan sebelumnya.

> - Cepat menemukan hasil optimal, sehingga biasanya lebih cepat mendapatkan konfigurasi yang baik.

> - Fleksibel, mampu menangani parameter diskrit (misalnya jumlah pohon pada Random Forest) maupun parameter kontinu (misalnya learning rate).

In [18]:
# import library
!pip install scikit-optimize

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [19]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.75, random_state=0
)

In [23]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(random_state=0))
])

In [20]:
# Definisikan Model dan parameter yang akan dioptimasi (Ruang Hyperparameter)
from skopt.space import Real, Integer, Categorical
search_space = {
    "clf__C": Real(1e-3, 1e3, prior="log-uniform"),
    "clf__gamma": Real(1e-4, 1e0, prior="log-uniform"),
}

# Fungsi Optimisasi Menggunakan BayesianSearchCV

In [21]:
from skopt.space import Real

search_space = {
    "clf__C": Real(1e-3, 1e3, prior="log-uniform"),
    "clf__gamma": Real(1e-4, 1e0, prior="log-uniform"),
}

In [24]:
# Inisialisasi BayesSearchCV
from skopt import BayesSearchCV

opt = BayesSearchCV(
    estimator=pipe,
    search_spaces=search_space,
    n_iter=20,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=0,
    verbose=0
)

In [25]:
# Jalankan optimisasi
opt.fit(X_train, y_train)

print("Best CV Accuracy :", opt.best_score_)
print("Best Parameters  :", opt.best_params_)


Best CV Accuracy : 0.9557312252964426
Best Parameters  : OrderedDict({'clf__C': 1.5338574134575653, 'clf__gamma': 0.05061952149969536})


In [26]:
# Evaluasi Model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = opt.predict(X_test)

print("Test Accuracy      :", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Test Accuracy      : 0.9736842105263158

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

Confusion Matrix:
 [[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]
